## Data Leakage, L1(Lasso) and L2 (Ridge) regularization using Linear Regression

We will use cross validation, lasso and ridge regression in this lab.

Specifically speaking, <br>
Regularization basically adds the penalty as model complexity increases.<br>
Cross validation is used to evaluate how well our model can generalize on the dataset. <br>

We will be using r2 score in this lab. It provides an indication of goodness of fit and therefore a measure of how well unseen samples are likely to be predicted by the model.


In this task, we will explore the following things on linear regression model:
- Cross Validation
- L1 regularization (Lasso regression)
- L2 regularization (Ridge regression)


#### Dataset
The dataset is available at "data/bike.csv" in the respective challenge's repo.<br>

The dataset is __modified version__ of the dataset 'bike.csv' provided by UCI Machine Learning repository.

Original dataset: https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

#### Objective
To learn about how cross validation, L1 regularization and L2 regularization work.

#### Tasks
- load the dataset.
- perform pre-processing on the data.
- remove registered feature and keep the casual feature to understand data leakage.
- construct train and test dataset.
- create a linear regression model.
- check the r2 score of the initial linear regression model on train and test dataset
- observe distribution of weights in the initial linear regression model. 
- split the dataset into k consecutive folds.
- calculate cross validation score for the k fold and check how well our model can generalize on the training dataset.
- checking the variance threshold of dataset and remove features with low variance.
- apply L1 regularization on the dataset and check the r2_score.
- visualize the distribution of weights on the lasso regression model.
- apply L2 regularization on the dataset and check the r2_score.
- visualize the distribution of weights on the ridge regression model. 

#### Further fun
- apply RFE on the dataset to automatically remove uneccessary features which would prevent overfitting.
- don't remove casual and registered features and check the effect of data leakage on the model
- implement lasso and ridge regression without using inbuilt librarires.
- apply elastic net to visualize the effect of both ridge and lasso regression.


#### Helpful links
- Cross validation : https://machinelearningmastery.com/k-fold-cross-validation/#:~:text=Cross%2Dvalidation%20is%20a%20resampling,k%2Dfold%20cross%2Dvalidation.
- Cross validation: https://scikit-learn.org/stable/modules/cross_validation.html
- L1 and L2 regularization : https://towardsdatascience.com/ridge-and-lasso-regression-a-complete-guide-with-python-scikit-learn-e20e34bcbf0b
- L1 and L2 regularization : https://www.youtube.com/watch?v=9lRv01HDU0s&list=PLZoTAELRMXVPBTrWtJkn3wWQxZkmTXGwe&index=30&t=904s
- r2_score: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html#sklearn.metrics.r2_score
- pd.get_dummies() and One Hot Encoding: https://queirozf.com/entries/one-hot-encoding-a-feature-on-a-pandas-dataframe-an-example
- Data Leakage : "https://machinelearningmastery.com/data-leakage-machine-learning/
- sklearn k-fold : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
- sklearn cross_val_score : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html?highlight=cross_val_score#sklearn.model_selection.cross_val_score
- sklearn lasso regression : https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html?highlight=lasso#sklearn.linear_model.Lasso
- sklearn ridge regression : https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html?highlight=ridge#sklearn.linear_model.Ridge
- RFE : https://machinelearningmastery.com/rfe-feature-selection-in-python/
- RFE sklearn : https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html
- Use slack for doubts: 

In [1]:
#import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Sklearn processing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# Sklearn linear regression model
from sklearn.linear_model import LinearRegression

# Sklearn regression model evaluation functions
from sklearn.metrics import r2_score

# Perform feature selection using a variance threshold
from sklearn.feature_selection import VarianceThreshold

# Feature selection using Recursive Feature Elimimation
from sklearn.feature_selection import RFE

In [2]:
#load the data and inspect the first 5 rows
data = pd.read_csv('data/bike.csv')
data.head()

,id,season,year,holiday,weekday,workingday,weather,temp,feel_temp,hum,windspeed,promotion_level,promotion_type,promotion_level_external,promotion_type_external,casual,registered,cnt
0,12765,winter,2016,0,6,0,cloud,0.344167,0.363625,0.805833,0.160446,7,1,2,2,226,654,880
1,12766,winter,2016,0,0,0,cloud,0.363478,0.353739,0.696087,0.248539,8,1,8,1,125,670,795
2,12767,winter,2016,0,1,1,fair,0.196364,0.189405,0.437273,0.248309,3,1,10,2,75,1229,1304
3,12768,winter,2016,0,2,1,fair,0.200000,0.212122,0.590435,0.160296,0,1,8,3,67,1454,1521
4,12769,winter,2016,0,3,1,fair,0.226957,0.229270,0.436957,0.186900,2,0,5,1,58,1518,1576


In [7]:
# print the data types of each feature name
a=data.columns
print(a)

Index(['id', 'season', 'year', 'holiday', 'weekday', 'workingday', 'weather',
       'temp', 'feel_temp', 'hum', 'windspeed', 'promotion_level',
       'promotion_type', 'promotion_level_external', 'promotion_type_external',
       'casual', 'registered', 'cnt'],
      dtype='object')


In [6]:
# check for null values in each column
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 18 columns):
id                          365 non-null int64
season                      365 non-null object
year                        365 non-null int64
holiday                     365 non-null int64
weekday                     365 non-null int64
workingday                  365 non-null int64
weather                     365 non-null object
temp                        365 non-null float64
feel_temp                   365 non-null float64
hum                         365 non-null float64
windspeed                   365 non-null float64
promotion_level             365 non-null int64
promotion_type              365 non-null int64
promotion_level_external    365 non-null int64
promotion_type_external     365 non-null int64
casual                      365 non-null int64
registered                  365 non-null int64
cnt                         365 non-null int64
dtypes: float64(4), int64(12), object(2

In [15]:
for i in a:
    print(data[i].unique())
    

[12765 12766 12767 12768 12769 12770 12771 12772 12773 12774 12775 12776
 12777 12778 12779 12780 12781 12782 12783 12784 12785 12786 12787 12788
 12789 12790 12791 12792 12793 12794 12795 12796 12797 12798 12799 12800
 12801 12802 12803 12804 12805 12806 12807 12808 12809 12810 12811 12812
 12813 12814 12815 12816 12817 12818 12819 12820 12821 12822 12823 12824
 12825 12826 12827 12828 12829 12830 12831 12832 12833 12834 12835 12836
 12837 12838 12839 12840 12841 12842 12843 12844 12845 12846 12847 12848
 12849 12850 12851 12852 12853 12854 12855 12856 12857 12858 12859 12860
 12861 12862 12863 12864 12865 12866 12867 12868 12869 12870 12871 12872
 12873 12874 12875 12876 12877 12878 12879 12880 12881 12882 12883 12884
 12885 12886 12887 12888 12889 12890 12891 12892 12893 12894 12895 12896
 12897 12898 12899 12900 12901 12902 12903 12904 12905 12906 12907 12908
 12909 12910 12911 12912 12913 12914 12915 12916 12917 12918 12919 12920
 12921 12922 12923 12924 12925 12926 12927 12928 12

In [16]:
# print out the value counts (frequency of occurence) of the unique values in these features ['season', 'year', 'weather', 'promotion_type']
data.apply(pd.Series.nunique)



id                          365
season                        4
year                          1
holiday                       2
weekday                       7
workingday                    2
weather                       3
temp                        294
feel_temp                   353
hum                         332
windspeed                   349
promotion_level              10
promotion_type                2
promotion_level_external     10
promotion_type_external       4
casual                      288
registered                  347
cnt                         345
dtype: int64

In [17]:
# print the shape of data
data.shape

(365, 18)

In [18]:
# drop the feature 'id' as it has no information to deliver.

data = data.drop('id', axis = 1)

In [19]:
# print the shape of data
data.shape

(365, 17)

In [20]:
# one hot encode the categorical columns.
data=pd.get_dummies(data, columns=["season","weather"])


In [21]:
# print the shape of data 
# notice the increase in the no. of features
data.shape

(365, 22)

Notice that our target feature "cnt" is the sum of the features "registered" + "casual"<br>

To avoid data leakage remove the feature "casual" for the training purpose. <br>

To understand more about data leakage refer the article mentioned in the uselful links.

In [ ]:
# Split the dataset into X and y
# While loading data into X drop the columns "cnt" and "casual". 
X = ?

# notice the target variable is 'cnt'
y = ?

In [ ]:
# store the names of the training features / name of the columns used for training. [Very important step for visualization later.]

train_columns = list(X.columns)
print(train_columns)

In [ ]:
# Apply scaling if our data is spread across wide differences of range values.


In [ ]:
# print the type of X

Note : <br>
Type of X should be pandas dataframe.
If not then convert X into pandas DataFrame object before proceeding further.


In [ ]:
# convert X into pandas Dataframe
# in the parameters specify columns = train_columns.

X = pd.DataFrame(X, columns = train_columns)
X.head()

In [ ]:
# split the dataset into X_train, X_test, y_train, y_test
# play around with test sizes.

test_size = ?
X_train, X_test, y_train, y_test = ?

In [ ]:
# print the shapes
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# build the Linear Regression model.

model = ?

# fit the model on the training data
model.fit(?, ?)

In [ ]:
# print the score on training set
y_pred_train = model.predict(?)
print("On Training set : ", r2_score(y_train, y_pred_train))

In [ ]:
# print the score on the test set
y_pred_test = model.predict(?)
print("On testing set : ", r2_score(?, ?))

Do not edit the code given below. Observe the distribution of weights. 
Which feature has the maximum coefficient ? <br>
Keep this figure as a base reference for visualizing the effects of l1-norm and l2-norm later in this notebook.

In [ ]:
# custom summary function to plot the coefficients / weightage of the features.
def custom_summary(model, column_names, title):
    '''Show a summary of the trained linear regression model'''

    # Plot the coeffients as bars
    fig = plt.figure(figsize=(8,len(column_names)/3))
    fig.suptitle(title, fontsize=16)
    rects = plt.barh(column_names, model.coef_,color="lightblue")

    # Annotate the bars with the coefficient values
    for rect in rects:
        width = round(rect.get_width(),4)
        plt.gca().annotate('  {}  '.format(width),
                    xy=(0, rect.get_y()),
                    xytext=(0,2),  
                    textcoords="offset points",  
                    ha='left' if width<0 else 'right', va='bottom')        
    plt.show()

In [ ]:
# coefficients plot
# let's call the above custom function.

custom_summary(model, train_columns, "Linear Regression coefficients.")

In [ ]:
# evaluate the model with k = 10 Fold Cross validation

folds = KFold(n_splits = 10, shuffle = True, random_state = 100)
results = cross_val_score(?, ?, ?, scoring = 'r2', cv = folds)

print(type(model).__name__)
print("kFoldCV:")
print("Fold R2 scores:", results)
print("Mean R2 score:", results.mean())
print("Std R2 score:", results.std())
print("Generalizability on training set : ", results.mean(), " +/- ", results.std())

Feature Selection using Variance Thresholding

In [ ]:
print("Original shape of X_train : ", X_train.shape)

In [ ]:
# check the variance of X.
# Note the type(X) should be a pandas DataFrame as stated earlier.

X.var()

Remove low variance features using Variance Threshold. 

Note : If the variance is less, it implies the values of that particular feature spans limited range of values.

In [ ]:
# play around with the threshold values

sel = VarianceThreshold(threshold = (0.01))
sel.fit(X_train)

In [ ]:
# do not edit.

selected_features = list(X_train.columns[sel.get_support()])
print("Selected features : ", selected_features)
print("Removed features : ", list(X_train.columns[~sel.get_support()]))

In [ ]:
# Delete the removed features from the train_columns list.

for i in removed_features:
  train_columns.remove(i)

In [ ]:
#transform / remove the low variance features

X_train = sel.transform(X_train)
X_test = sel.transform(X_test)

## Lasso Regression : L1 - norm

In [ ]:
from sklearn.linear_model import Lasso

X_train, X_test, y_train, y_test = train_test_split(?, ?, test_size = ?, random_state = 100)

# hyperparamater alpha : controls the degree of penaliation.
# play around with alpha values.
alpha = 1.0

#create the model
model_lasso = Lasso(alpha = alpha)

#fit the model on training data
model_lasso.fit(?, ?)

#calculate the score on training data
y_pred_train = model_lasso.predict(?)
print("On train set : ", r2_score(y_train, y_pred_train))

In [ ]:
#evaluate the model on testing data
y_pred_test = model_lasso.predict(?)
print("On test set : ", r2_score(y_test, ?))

In [ ]:
# visualize the coefficients.
# compare the results with the plot obtained earlier.

custom_summary(model_lasso, train_columns, "Lasso Regression Coefficients.")

We can see that Lasso regression has automatically done a lot of feature selection. Some columns might have zero coefficients. It has been effectively removed. <br> 
The model is much more interpretable than the baseline linear regression model.
<br>
Hence, Lasso regression has embedded Feature Selection.

# Ridge Regression : L2 - norm

In [ ]:
from sklearn.linear_model import Ridge

# hyperparamater alpha : controls the degree of penaliation.
# play around with alpha values.
alpha = 1.0

#create the model
model_ridge = Ridge(alpha = ?)

#fit the model on training data
model_ridge.fit(?, ?)

#calculate the score on training data
y_pred_train = model_ridge.predict(?)
print("On train set : ", r2_score(y_train, ?))


In [ ]:
#evaluate the model on testing data
y_pred_test = model_ridge.predict(?)
print("On test set : ", r2_score(y_test, ?))

In [ ]:
# visualize the coefficients.
# compare the results with the plot obtained earlier.

custom_summary(model_ridge, train_columns, "Ridge Regression Coefficients.")

Ridge regression doesn't drive smaller coefficients to 0 hence it doesn't possess internal feature selection.

Points to Ponder ! [Optional]

Did you notice the highest dependency on the feature "registered" if you haven't removed it till now ?

Since our target is "cnt" which is the simple combination of "registered" and "casual".

we have removed "casual", but the model was smart enough to predict the target "cnt" simply from one feature "registered" itself. 

This is the classic example of Data Leakage. So the aim here is not to make 99 percent accurate predictions, the aim is to take into account the factors for making predictions.

So, to get a detailed report, we should avoid data leakage thereby removing both the features "registered" and "casual".